In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [3]:
# Download, load, extract, model...
interval = '1d'
stock_set = f.download_interval_process(interval, 1)
print(f'Stock set: \n{stock_set}\n')

Stock set: 
{'WELL', 'CMCSA', 'NVDA', 'BAC', 'XEL', 'UNH', 'AEP', 'CSCO', 'NFLX', 'RCL', 'AJG', 'GOOGL', 'APH', 'ATO', 'PLTR', 'KMI', 'LNG', 'XOM', 'IBM', 'JPM', 'AAPL', 'JNJ', 'FCNCA', 'FWONK', 'WMB', 'WSM', 'NRG', 'MSI', 'DASH', 'HD', 'ACN', 'MSFT', 'ISRG', 'LLY', 'EXE', 'D', 'WMT', 'TGT', '^VIX', 'MRK', 'ABBV', 'AMGN', 'KO', 'OKE', 'EXC', 'CEG', 'BRK-B', 'AVGO', 'COP', 'MO', 'ORCL', 'DUK', 'SRE', 'COST', 'TDG', 'PGR', 'SLB', 'PG', 'CL', 'TPR', 'CVX', 'TMUS', 'ADBE', 'AXP', 'PSX', 'TSLA', 'MA', 'VZ', 'META', 'VST', 'GOOG', 'NEE', 'DIS', 'ABT', 'T', 'MDLZ', 'DKNG', 'DECK', 'SO', 'SPGI', 'PEP', 'WFC', 'EOG', 'PM', 'BSX', 'TMO', 'TTWO', 'V', 'CRM', 'GS', 'AMZN', 'SW'}



In [4]:
f.make_table_features_process(stock_set, interval, 8)

In [5]:
f.make_master_table(stock_set, interval)

Saved combined dataframe to: ./data_transformed/all_1d_model_df.pkl


In [6]:
f.xg_boost_model(interval='1d', grid_search_on=True)

Best parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__reg_alpha': 0.0, 'classifier__reg_lambda': 1.5, 'feature_selection__threshold': 0.005}
              precision    recall  f1-score   support

           0       0.38      0.40      0.39     55500
           1       0.39      0.41      0.40     55430
           2       0.40      0.36      0.38     55424

    accuracy                           0.39    166354
   macro avg       0.39      0.39      0.39    166354
weighted avg       0.39      0.39      0.39    166354


Feature Importances (Descending):
                   feature  importance
17     cat__day_of_month_4    0.031277
40    cat__day_of_month_27    0.028218
36    cat__day_of_month_23    0.027062
21     cat__day_of_month_8    0.026941
2     cat__month_of_year_1    0.025555
37    cat__day_of_month_24    0.025341
13   cat__month_of_year_12    0.024583
31    cat__day_of_month_18    0.024489
34    cat__day_of_mo

In [7]:
symbol = 'TSLA'.upper()
interval = '1d'
f.download(symbol='^VIX', interval=interval)
f.model_prospect(symbol, interval)
df_val = f.model_validation(symbol, interval)
correctly_predicted = df_val.dir_pred_match.sum()
print(f'\nCorrectly Predicted: {correctly_predicted}')
entries_predicted = df_val.shape[0]
print(f'Entries Predicted: {entries_predicted}')
print(f'Percent Correct: {correctly_predicted / entries_predicted}')


Model Prediction 1d TSLA:
Predicted Next 1d Movement: DOWN

Model Prediction Probabilities:
no_change (0): 0.2796
up (1): 0.3470
down (2): 0.3734

Last Entry 1d TSLA Datetime Used for Prediction:
NOTE: It's in or contains the full 1d time interval.
EST: 2025-01-23 12:00:00 AM EST-0500
PDT: 2025-01-22 09:00:00 PM PST-0800

Correctly Predicted: 13
Entries Predicted: 50
Percent Correct: 0.26
